In [1]:
import sqlite3
import pandas as pd
import io
import re

In [2]:
HOMEDIR='/home/jovyan/work/'

In [11]:
def find_tables(dump_filename):
    table_list=[]

    with open(dump_filename, 'r') as f:
        for line in f:
            line = line.strip()
            if line.lower().startswith('create table'):
                table_name = re.findall('`([\w_]+)`', line)
                table_list.append(table_name[0])

    return table_list

def read_dump(dump_filename, target_table):
    with io.StringIO() as sio:
        fast_forward = True
        bulk = False
        with open(dump_filename, 'r') as f:
            for line in f:
                line = line.strip()
        
                if line.lower().startswith('insert') and target_table in line.replace('`', '').split():
                    fast_forward = False
                    if not bulk:
                        data = re.findall('\([^\)]*\)', line)
                        newline = data[0].strip(' ()').replace('`', '')
                        sio.write(newline)
                        sio.write("\n")
                    bulk=True
                    continue
                if fast_forward:
                    continue
                if line.endswith(';'):
                    fast_forward=True
                try:
                    newline = line.strip('(').rstrip('),').rstrip(');')
                    newline = newline.strip(' ()').replace('\'', '"').replace('NULL', '')
                    sio.write(newline)
                    sio.write("\n")
                except IndexError:
                    pass
                except:
                    print(line)
                    print(newline)
                    print('////////////////////////////////////////////////////////////////////')
                    
                    pass
                
        sio.seek(0)

        try:
            with open(f'{HOMEDIR}datasets/waterbirds/{target_table}.csv', 'w') as archive:
                  archive.write(sio.getvalue())
            return pd.read_csv(sio, quotechar = '"', skipinitialspace=True)
        except Exception as e:
            print(target_table)
            print('____________________')
            print(e)
            

In [12]:
dataDump=f'{HOMEDIR}datasets/waterbirds/wetlands_wpe.sql'
tables = []
tableNames=find_tables(dataDump)

for table in tableNames:
    data = read_dump(dataDump, table)
    tables.append(data)

PopulationName
____________________
Error tokenizing data. C error: Expected 17 fields in line 517, saw 22

PopulationNameBackup
____________________
Error tokenizing data. C error: Expected 17 fields in line 517, saw 22

PopulationSize
____________________
Error tokenizing data. C error: Expected 9 fields in line 2766, saw 10

PopulationTrend
____________________
Error tokenizing data. C error: Expected 8 fields in line 4387, saw 9

Reference
____________________
Error tokenizing data. C error: Expected 4 fields in line 113, saw 7

user_populationname
____________________
No columns to parse from file


### **note:** to have in mind we will need to set the constrains to mimic the mysql db.

--
-- Constraints for dumped tables
--

--
-- Constraints for table `Family`
--
ALTER TABLE `Family`
  ADD CONSTRAINT `FK_A24A246D5DDE227E` FOREIGN KEY (`grouping_id`) REFERENCES `FamilyOrder` (`id`);

--
-- Constraints for table `PopulationName`
--
ALTER TABLE `PopulationName`
  ADD CONSTRAINT `FK_C20E5FB6A865C3C` FOREIGN KEY (`flyway_id`) REFERENCES `Flyway` (`id`),
  ADD CONSTRAINT `FK_C20E5FB6B2A1D860` FOREIGN KEY (`species_id`) REFERENCES `Species` (`id`);

--
-- Constraints for table `PopulationOnePercentLevel`
--
ALTER TABLE `PopulationOnePercentLevel`
  ADD CONSTRAINT `FK_A5E8AE5234B2DE5B` FOREIGN KEY (`populationId`) REFERENCES `PopulationName` (`id`);

--
-- Constraints for table `PopulationProtection`
--
ALTER TABLE `PopulationProtection`
  ADD CONSTRAINT `FK_F7DF0C0367A8D7E6` FOREIGN KEY (`protection_id`) REFERENCES `Protection` (`id`),
  ADD CONSTRAINT `FK_F7DF0C03C955D1E1` FOREIGN KEY (`population_id`) REFERENCES `PopulationName` (`id`);

--
-- Constraints for table `PopulationPublication`
--
ALTER TABLE `PopulationPublication`
  ADD CONSTRAINT `FK_8D7E4BE338B217A7` FOREIGN KEY (`publication_id`) REFERENCES `Publication` (`id`),
  ADD CONSTRAINT `FK_8D7E4BE33EBA329F` FOREIGN KEY (`onePercent_id`) REFERENCES `PopulationOnePercentLevel` (`id`),
  ADD CONSTRAINT `FK_8D7E4BE36B302DC7` FOREIGN KEY (`populationSize_id`) REFERENCES `PopulationSize` (`id`),
  ADD CONSTRAINT `FK_8D7E4BE3C598F54F` FOREIGN KEY (`populationTrend_id`) REFERENCES `PopulationTrend` (`id`),
  ADD CONSTRAINT `FK_8D7E4BE3C955D1E1` FOREIGN KEY (`population_id`) REFERENCES `PopulationName` (`id`);

--
-- Constraints for table `PopulationSize`
--
ALTER TABLE `PopulationSize`
  ADD CONSTRAINT `FK_6BED05DA92B924B6` FOREIGN KEY (`estimateQuality_id`) REFERENCES `QualityCodeSize` (`id`),
  ADD CONSTRAINT `FK_6BED05DAC955D1E1` FOREIGN KEY (`population_id`) REFERENCES `PopulationName` (`id`);

--
-- Constraints for table `PopulationSizeReferenceCode`
--
ALTER TABLE `PopulationSizeReferenceCode`
  ADD CONSTRAINT `FK_2340E9DF1645DEA9` FOREIGN KEY (`reference_id`) REFERENCES `Reference` (`id`),
  ADD CONSTRAINT `FK_2340E9DF6B302DC7` FOREIGN KEY (`populationSize_id`) REFERENCES `PopulationSize` (`id`);

--
-- Constraints for table `PopulationTrend`
--
ALTER TABLE `PopulationTrend`
  ADD CONSTRAINT `FK_3F06AD0B65B0AAB2` FOREIGN KEY (`trend_id`) REFERENCES `Trend` (`id`),
  ADD CONSTRAINT `FK_3F06AD0B726BFED0` FOREIGN KEY (`trendQuality_id`) REFERENCES `QualityCodeTrend` (`id`),
  ADD CONSTRAINT `FK_3F06AD0BC955D1E1` FOREIGN KEY (`population_id`) REFERENCES `PopulationName` (`id`);

--
-- Constraints for table `PopulationTrendReferenceCode`
--
ALTER TABLE `PopulationTrendReferenceCode`
  ADD CONSTRAINT `FK_9D27E20B1645DEA9` FOREIGN KEY (`reference_id`) REFERENCES `Reference` (`id`),
  ADD CONSTRAINT `FK_9D27E20BC598F54F` FOREIGN KEY (`populationTrend_id`) REFERENCES `PopulationTrend` (`id`);

--
-- Constraints for table `Species`
--
ALTER TABLE `Species`
  ADD CONSTRAINT `FK_6AB2CE8E8EA915DC` FOREIGN KEY (`familyName_id`) REFERENCES `Family` (`id`),
  ADD CONSTRAINT `FK_6AB2CE8EED246725` FOREIGN KEY (`IUCN_id`) REFERENCES `RedListCategory` (`id`);

--
-- Constraints for table `user_flyway`
--
ALTER TABLE `user_flyway`
  ADD CONSTRAINT `FK_4449CE28A76ED395` FOREIGN KEY (`user_id`) REFERENCES `user` (`id`) ON DELETE CASCADE,
  ADD CONSTRAINT `FK_4449CE28A865C3C` FOREIGN KEY (`flyway_id`) REFERENCES `Flyway` (`id`) ON DELETE CASCADE;

--
-- Constraints for table `user_populationname`
--
ALTER TABLE `user_populationname`
  ADD CONSTRAINT `FK_C4DD57099C1720AA` FOREIGN KEY (`populationname_id`) REFERENCES `PopulationName` (`id`) ON DELETE CASCADE,
  ADD CONSTRAINT `FK_C4DD5709A76ED395` FOREIGN KEY (`user_id`) REFERENCES `user` (`id`) ON DELETE CASCADE;